In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -qq install OSKut
# !pip -qq install sefr_cut

In [ ]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import sys
import oskut
import numpy as np
import time
from tqdm import tqdm
from sklearn.metrics import f1_score
import json

In [ ]:
!gdown --id '11EAAsZ3ofLUOT5UIu85PTTkSvO-DKdhP' # WS_TEST
!gdown --id '1UDtrQ3UgaX7f8Lu9rptkPUly7_46U9Em' # NE_TEST

Downloading...
From: https://drive.google.com/uc?id=11EAAsZ3ofLUOT5UIu85PTTkSvO-DKdhP
To: /content/ws_test.txt
100% 106k/106k [00:00<00:00, 82.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UDtrQ3UgaX7f8Lu9rptkPUly7_46U9Em
To: /content/ne_test.txt
100% 789k/789k [00:00<00:00, 162MB/s]


In [ ]:
def readtxt(path):
    test_data = open(path, 'r')
    return test_data.readlines()

In [ ]:
def char_split_word_tag(df):
    list_word = df['word'].tolist()
    word_tag = []
    for word in list_word:
        if word == ' ':
            word_tag.append(('', 'O'))
            continue
        for c in word:
            if c == '"' or c == '':
                word_tag.append((c, 'I_WORD'))
                continue
            elif c==' ':
                word_tag.append(('', 'O'))
            elif c == word[0]:
                word_tag.append((c, 'B_WORD'))
            elif c == word[-1]:
                word_tag.append((c, 'E_WORD'))
            else:
                word_tag.append((c, 'I_WORD'))
    return pd.DataFrame(word_tag, columns=['Id', 'Predicted'])

In [ ]:
# text = readtxt('/content/ws_test.txt')
textNE = readtxt('/content/ne_test.txt')

In [ ]:
paragraph = ''.join(textNE).replace('\n\n', ' ').replace('\n', '')

In [ ]:
textNE_replace = []
for t in textNE:
    if t == '\n':
        textNE_replace.append(' ')
    else: 
        textNE_replace.append(t.replace('\n',''))

print(textNE_replace)

['นาง', 'สุดารัตน์', ' ', 'เกยุราพันธ์', ' ', 'รมว.', 'สาธารณสุข', ' ', 'แถลง', 'ถึง', 'การ', 'จัด', 'งาน', 'มหกรรม', 'กิน', 'ไก่', 'ไทย', 'ปลอด', 'ภัย', ' ', '100', '%', ' ', 'ว่า', ' ', 'รัฐบาล', 'ร่วม', 'กับ', 'กทม.', 'จัด', 'งาน', 'ดัง', 'กล่าว', 'ขึ้น', 'ที่', 'ท้องสนามหลวง', 'ใน', 'วัน', 'เสาร์', 'ที่', ' ', '7', ' ', 'ก.พ.', 'นี้', ' ', 'ตั้งแต่', ' ', '16.00', 'น.', 'เป็นต้นไป', ' ', 'ส่วน', 'พ.ต.ท.', 'ทักษิณ', ' ', 'ชินวัตร', 'รัฐมนตรี', ' ', 'จะ', 'เดินทาง', 'มา', 'ถึง', 'ที่', 'งาน', 'ใน', 'เวลา', ' ', '16.30', 'น.', ' ', 'เพื่อ', 'เยี่ยมชม', 'นิทรรศการ', ' ', 'และ', 'ประกอบ', 'อาหาร', 'เมนู', ' ', 'กระเพรา', 'ไก่', 'ไข่ดาว', 'สุก', ' ', 'ไก่', 'ทอด', ' ', 'ด้วย', 'ตัว', 'เอง', 'และ', 'แจกจ่าย', 'ให้', 'กับ', 'ประชาชน', ' ', 'โดย', 'จะ', 'มี', 'นาย', 'สมัคร', ' ', 'สุนทรเวช', ' ', 'ผู้', 'ว่า', 'ฯ', 'กทม.', ' ', 'และ', 'ม.ล.', 'ศิริเฉลิม', ' ', 'สวัสดิวัฒน์', ' ', 'หรือ', 'หมึกแดง', ' ', 'ร่วม', 'ปรุง', 'อาหาร', 'ด้วย', ' ', 'ภาย', 'หลัง', 'ที่', 'นายก', 'ฯ', 'ปรุง', 'อาหาร'

In [ ]:
df_tag = pd.DataFrame(textNE_replace, columns=['word']) 
df_tag_char = char_split_word_tag(df_tag).to_numpy()

In [ ]:
print(textNE_replace)

['นาง', 'สุดารัตน์', ' ', 'เกยุราพันธ์', ' ', 'รมว.', 'สาธารณสุข', ' ', 'แถลง', 'ถึง', 'การ', 'จัด', 'งาน', 'มหกรรม', 'กิน', 'ไก่', 'ไทย', 'ปลอด', 'ภัย', ' ', '100', '%', ' ', 'ว่า', ' ', 'รัฐบาล', 'ร่วม', 'กับ', 'กทม.', 'จัด', 'งาน', 'ดัง', 'กล่าว', 'ขึ้น', 'ที่', 'ท้องสนามหลวง', 'ใน', 'วัน', 'เสาร์', 'ที่', ' ', '7', ' ', 'ก.พ.', 'นี้', ' ', 'ตั้งแต่', ' ', '16.00', 'น.', 'เป็นต้นไป', ' ', 'ส่วน', 'พ.ต.ท.', 'ทักษิณ', ' ', 'ชินวัตร', 'รัฐมนตรี', ' ', 'จะ', 'เดินทาง', 'มา', 'ถึง', 'ที่', 'งาน', 'ใน', 'เวลา', ' ', '16.30', 'น.', ' ', 'เพื่อ', 'เยี่ยมชม', 'นิทรรศการ', ' ', 'และ', 'ประกอบ', 'อาหาร', 'เมนู', ' ', 'กระเพรา', 'ไก่', 'ไข่ดาว', 'สุก', ' ', 'ไก่', 'ทอด', ' ', 'ด้วย', 'ตัว', 'เอง', 'และ', 'แจกจ่าย', 'ให้', 'กับ', 'ประชาชน', ' ', 'โดย', 'จะ', 'มี', 'นาย', 'สมัคร', ' ', 'สุนทรเวช', ' ', 'ผู้', 'ว่า', 'ฯ', 'กทม.', ' ', 'และ', 'ม.ล.', 'ศิริเฉลิม', ' ', 'สวัสดิวัฒน์', ' ', 'หรือ', 'หมึกแดง', ' ', 'ร่วม', 'ปรุง', 'อาหาร', 'ด้วย', ' ', 'ภาย', 'หลัง', 'ที่', 'นายก', 'ฯ', 'ปรุง', 'อาหาร'

In [ ]:
# engine_list = ['tl-deepcut-lst20','ws', 'ws-augment-60p', 'tnhc', 'best', 'scads', 'tl-deepcut-ws', 'tl-deepcut-tnhc', 'deepcut']
engine_list = ['tl-deepcut-ws', 'tl-deepcut-tnhc', 'deepcut','scads']   #'best' ใช้ไม่ได้, 'scads' นานมาก
f1_dict = {}
time_dict = {}
for eng in tqdm(engine_list):
    oskut.load_model(engine=eng)
    tic = time.time()
    # print(tic)
    pred = oskut.OSKut(paragraph)
    toc = time.time()
    pred_df = pd.DataFrame(pred, columns=['word']) 
    pred_char = char_split_word_tag(pred_df).to_numpy()
    f1 = f1_score(df_tag_char[:,1], pred_char[:,1], average='macro')
    f1_dict[eng] = f1
    time_dict[eng] = toc-tic
    with open('/content/drive/MyDrive/SuperAI2-946/onsite/hackathon/week45_NLP/time_dict2.json', 'w') as f:
        json.dump(time_dict, f)
    with open('/content/drive/MyDrive/SuperAI2-946/onsite/hackathon/week45_NLP/f1_dict2.json', 'w') as f:
        json.dump(f1_dict, f)

  0%|          | 0/5 [00:00<?, ?it/s]

ws


 20%|██        | 1/5 [00:31<02:06, 31.57s/it]

tnhc


 60%|██████    | 3/5 [01:35<01:03, 31.72s/it]


Exception: ignored

In [ ]:
f1_dict

{'deepcut': 0.9687740791594902,
 'tl-deepcut-tnhc': 0.9528560705149149,
 'tl-deepcut-ws': 0.9681829659923843}

In [ ]:
time_dict

{'deepcut': 31.099127292633057,
 'tl-deepcut-tnhc': 28.19572639465332,
 'tl-deepcut-ws': 29.319398641586304}

In [ ]:
f1_dict